<a href="https://colab.research.google.com/github/devvats/Competitions/blob/master/Code_Marathon_Round1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import cv2
import zipfile
from google.colab import drive
drive.mount('/content/drive')
import shutil

# Loading Data

In [0]:
#@title Default title text
shutil.copy('drive/My Drive/code marathon dataset/data/train.csv','./train.csv')
data = pd.read_csv("train.csv")
y = np.array(data["n_sides"])
x = list(data["image_name"])

In [0]:

Xtrain = x[:16000]
Ytrain = y[:16000]
Xval = x[16000:]
val = y[16000:]
train = Generator(Xtrain, Ytrain,8,10)
validation = Generator(Xval, Yval, 8, 10)

# Acc_criteria and Generator Function


In [0]:
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.sqrt(K.mean((y_pred-y_true)*(y_pred-y_true)))



In [0]:
from keras.utils import Sequence,to_categorical

class Generator(Sequence):
  
    def __init__(self,X,Y,batch,n_classes):
      
        self.x=X
        self.y=Y
        self.batch=batch
        self.n_classes=n_classes
        self.cnt=0
    
    def __len__(self):
       
        return int(np.floor(np.array(self.x).shape[0]) / self.batch)
  
  
    def __getitem__(self,index):
        
        path_x=self.x[index*self.batch:(index+1)*self.batch]
        path_y=self.y[index*(self.batch):(index+1)*self.batch]
        X=[]
        Y=path_y
       
        for x_path in path_x:
            temp_x= cv2.imread("drive/My Drive/code marathon dataset/data/train/"+x_path,0) 
            temp_x = x_path
            v = np.median(temp_x)
            lower = int(max(0, (1.0 - 0.33) * v))
            upper = int(min(255, (1.0 + 0.33) * v))
            edged = cv2.Canny(temp_x, lower, upper)
            temp_x = np.array(edged).reshape(256,256,1)
            temp_x = np.concatenate((temp_x, temp_x,temp_x), axis=2)
            
            self.cnt=self.cnt+1
            X.append(temp_x)
        X=np.array(X)
        Y=np.array(Y)
        return X,to_categorical(Y,self.n_classes)

# Model

In [0]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D
np.random.seed(17)

base_model = InceptionResNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dense(200, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

In [0]:
from keras import optimizers
ad = optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=ad, metrics=['accuracy',mean_pred])
#model.load_weights("drive/My Drive/code marathon dataset/data/model313.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=ad, metrics=['accuracy',mean_pred])

In [0]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
model_checkpoint = ModelCheckpoint("drive/My Drive/code marathon dataset/data/model313.hdf5", monitor='mean_pred', save_best_only=True,save_weights_only=True, verbose=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='mean_pred', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.000000000001)

In [0]:
Process = model.fit_generator(generator=train,
                    validation_data=validation,
                    use_multiprocessing=True,
                    workers = 4,
                    epochs = 15,
                    verbose = 1,
                    shuffle = True,
                    callbacks=[model_checkpoint,learning_rate_reduction] )

In [0]:
#model.save_weights("drive/My Drive/code marathon dataset/data/model311.h5")
model.save_weights("drive/My Drive/code marathon dataset/data/modelabc.h5")